In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
import copy
import seaborn as sns
import tensorflow as tf
from sklearn.linear_model import LinearRegression

dataset from https://archive.ics.uci.edu/ml/datasets/Seoul+Bike+Sharing+Demand#

Data Source:
http://data.seoul.go.kr/
SOUTH KOREA PUBLIC HOLIDAYS. URL: publicholidays.go.kr

In [2]:
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv

--2023-04-02 11:17:03--  https://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 604166 (590K) [application/x-httpd-php]
Saving to: ‘SeoulBikeData.csv’

SeoulBikeData.csv   100%[===================>] 590.01K   640KB/s    in 0.9s    

2023-04-02 11:17:04 (640 KB/s) - ‘SeoulBikeData.csv’ saved [604166/604166]



In [3]:
dataset_cols = ["bike_count", "hour", "temp", "humidity", "wind", "visibility", "dew_pt_temp", "radiation", "rain", "snow", "functional"]

In [7]:
df = pd.read_csv("SeoulBikeData.csv")